# Documentação

`!pip install yfinance`

### Artigo original:
https://towardsdatascience.com/how-to-create-a-fully-automated-ai-based-trading-system-with-python-708503c1a907

### Modelo ARIMA:
https://pt.wikipedia.org/wiki/ARIMA

### Uso do Auto-ARIMA:
https://towardsdatascience.com/time-series-forecasting-using-auto-arima-in-python-bb83e49210cd



## Requisitos
~~~python
!pip install yfinance
~~~

In [1]:
from functions import test_unit_root, stationary_test

#colocar essa parada no functions!!!
def get_arima_model(s):
    
    is_stat = stationary_test(s)
    is_seasonal = test_unit_root(s) 
    arima_model = auto_arima(s, stationary=is_stat, start_p=0, d=1, start_q=0, max_p=5, max_d=5, max_q=5, start_P=0, D=1, 
                             start_Q=1, max_P=5, max_D=5, max_Q=5, m=12, seasonal=is_seasonal, error_action='warn', trace=True, 
                             suppress_warnings=True, stepwise=False, random_state=20, n_fits=50, n_jobs=-1)
    return arima_model

In [2]:
ticker = "RNDP11"
yticker = "RNDP11.SA"
n_steps = 30

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import os
import json
import datetime

from functions import get_finance_data, arima_forecast, get_arima_model

plt.style.use('fivethirtyeight')

# criando a pasta de registro de calibração
if not os.path.isdir("files"):
    os.mkdir("files")

In [ ]:
data = get_finance_data(yticker)
train = data['Low'][:len(data)-n_steps+1]
test = data['Low'][-n_steps:]

In [ ]:
with open('files/db.json') as json_file:
    jdata = json.load(json_file)

In [ ]:
if os.path.isfile("files/db.json"):
    with open('files/db.json','r+') as jfile:
        jdata = json.load(jfile)
        if ticker not in jdata.keys():
            print("ticker não encontrado")
            jdata[ticker]={}
            jdata[ticker].update({"yticker":yticker})
            jfile.seek(0)
            json.dump(jdata, jfile)            
jdata

In [ ]:
my_arima = []

if "ARIMA" in jdata[ticker]:
    print("carregando "+ticker+" do JSON")
    my_arima.append(jdata[ticker]["ARIMA"]["parametros"][0])
    my_arima.append(jdata[ticker]["ARIMA"]["parametros"][1])
    my_arima.append(jdata[ticker]["ARIMA"]["parametros"][2])
    my_arima.append(jdata[ticker]["ARIMA"]["parametros"][3])
    my_arima.append(jdata[ticker]["ARIMA"]["parametros"][4])
    my_arima.append(jdata[ticker]["ARIMA"]["parametros"][5])
    my_arima.append(jdata[ticker]["ARIMA"]["parametros"][6])
else:
    print("rodando auto arima para "+ticker)
    arima_model = get_arima_model(train)
    my_arima.append(arima_model.order[0])
    my_arima.append(arima_model.order[1])
    my_arima.append(arima_model.order[2])
    my_arima.append(arima_model.seasonal_order[0])
    my_arima.append(arima_model.seasonal_order[1])
    my_arima.append(arima_model.seasonal_order[2])
    my_arima.append(arima_model.seasonal_order[3])

In [ ]:
#inserir código pra rodar e capturar parametros do tensor flow
if "TensorFlow" in jdata[ticker]:
    print(jdata[ticker]["TensorFlow"]["parametros"])
else:
    print('rodar Tensor Flow '+ticker)

In [ ]:
with open('files/db.json','r+') as jfile:
    hoje = datetime.datetime.now().strftime("%d-%m-%Y")
    jdata[ticker].update({"ARIMA":{"parametros":my_arima,"train_date":hoje}})
    jfile.seek(0)
    json.dump(jdata, jfile)

In [ ]:
jdata

In [ ]:
def desvio(registro):
    return ((registro['predição']/registro['valor'])-1)*100

In [ ]:
df_arima = arima_forecast(train, next=n_steps, p=my_arima[0], d=my_arima[1], q=my_arima[2])

In [ ]:
df_arima

In [ ]:
df = pd.DataFrame({'ARIMA':df_arima,'valor':test}, index=test.index)
prd = df
df

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label="treino")
plt.plot(test, label="teste")
plt.plot(prd['ARIMA'], label="ARIMA")
plt.legend(loc='lower right')
plt.title(label=ticker)
plt.show

In [ ]:
test

In [ ]:
train[:-1]